In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import json
import jsonlines
import re
import numpy as np

In [5]:
ed_data=pd.read_csv("data/base_juguete - Educacion.tsv", sep='\t',na_values=None, keep_default_na=False)
ha_data=pd.read_csv("data/base_juguete - Habilidades.tsv", sep='\t',na_values=None, keep_default_na=False)
la_data=pd.read_csv("data/base_juguete - idioma.tsv", sep='\t',na_values=None, keep_default_na=False)
ot_data=pd.read_csv("data/base_juguete - otros.tsv", sep='\t',na_values=None, keep_default_na=False)
pe_data=pd.read_csv("data/base_juguete - person.tsv", sep='\t',na_values=None, keep_default_na=False)
tr_data=pd.read_csv("data/base_juguete - Trabajo.tsv", sep='\t',na_values=None, keep_default_na=False)
pem_data = ot_data.merge(pe_data, on='uid')

In [6]:
####PURAS ENTIDADES

class persona(object):
    uid=""
    nombre=""
    ubicacion=""
    id=""
    

class idiomas(object):
    uid=""
    idioma=""
    IDI_NIVEL_SP=""
    IDI_NIVEL_WR=""
    IDI_NIVEL_TR=""
    IDI_NIVEL_TE=""
    
    def __init__(self, idioma, IDI_NIVEL_SP,IDI_NIVEL_WR,IDI_NIVEL_TR,IDI_NIVEL_TE):
        self.idioma = idioma        
        self.IDI_NIVEL_SP=IDI_NIVEL_SP
        self.IDI_NIVEL_WR=IDI_NIVEL_WR
        self.IDI_NIVEL_TR=IDI_NIVEL_TR
        self.IDI_NIVEL_TE=IDI_NIVEL_TE
        
        
        
    def __json__(self):
        return {'IDI_NOMBRE':self.idioma, 'IDI_NIVEL_SP': self.IDI_NIVEL_SP,
                'IDI_NIVEL_WR':self.IDI_NIVEL_WR, 'IDI_NIVEL_TR':self.IDI_NIVEL_TR,
                'IDI_NIVEL_TE':self.IDI_NIVEL_TE }

class habilidades(object):
    ##tipo=""
    habilidades=""
    
    def __init__(self, habilidades):
        ##self.tipo = tipo
        self.habilidades = habilidades
        
    def __json__(self):
        return {'HAB_HABILIDAD':self.habilidades}


class educacion(object):
    uid=""
    institucion=""
    area=""
    fecha_inicio=""
    fecha_fin=""
    titulo=""
    extra_curriculares=""
    
    def __init__(self,institucion, area,  titulo):        #fecha_inicio, fecha_fin,
        self.institucion=institucion
        self.area=area
        #self.fecha_inicio=fecha_inicio
        #self.fecha_fin=fecha_fin
        self.titulo=titulo
        #self.extra_curriculares=extra_curriculares
        
    def __json__(self):
        return {'EDU_INST':self.institucion, 'EDU_AREA':self.area, 'EDU_TITULO':self.titulo}
        
class trabajos(object):
    uid=""
    titulo_posicion=""
    resumen=""
    fecha_inicio=""
    fecha_fin=""
    actual=""
    empresa=""
    area=""
    tipo=""
    industria=""
    voluntariado=""
    
    def __init__(self, titulo_posicion, empresa, area, tipo, industria, voluntariado):
        self.titulo_posicion=titulo_posicion
        #self.resumen=resumen
        #self.fecha_inicio=fecha_inicio
        #self.fecha_fin=fecha_fin
        #self.actual=actual
        self.empresa=empresa
        self.area=area
        self.tipo=tipo
        self.industria=industria
        #self.voluntariado=voluntariado
        
    def __json__(self):
        return {'TRA_POSICION': self.titulo_posicion, 'TRA_EMPRESA': self.empresa, 
                'TRA_AREA':self.area, 'TRA_TIPO':self.tipo, 'TRA_IND':self.industria}
        
class talento(object):
    uid=''
    nombre=''
    ubicacion=''
    id=''
    trabajos=[]
    cursos = []
    educacion = []
    certificaciones = []
    habilidades = []
    idiomas = []
    patentes = []
    publicaciones = []    
    
    def __init__(self, uid, nombre, ubicacion, id):
        self.uid = uid
        self.nombre = nombre
        self.ubicacion = ubicacion
        self.id = id
        
    def __json__(self):
        return {'uid': self.uid, 'nombre': self.nombre, 'ubicacion':self.ubicacion ,'id':self.id, 'trabajos':self.trabajos, 
                'educacion':self.educacion, 'habilidades':self.habilidades,
               'idiomas':self.idiomas}
    

class MyEncoder(json.JSONEncoder):
    """
    JSONEncoder subclass that leverages an object's `__json__()` method,
    if available, to obtain its default JSON representation. 

    """
    def default(self, obj):
        if hasattr(obj, '__json__'):
            return obj.__json__()
        return json.JSONEncoder.default(self, obj)
        
    

In [7]:
'''
HELPERS : 
'''
def parse_doc_list_plain(doc):        
    '''Preprocesing method, this code dont remove the character with graphic accent.'''
    red = ur'[^,a-z\u00E7\u00F1\u00E1\u00E9\u00ED\u00F3\u00FA\u00E0\u00E8\u00EC\u00F2\u00F9\u00E4\u00EB\u00EF\u00F6\u00FC\u00E2\u00EA\u00EE\u00F4\u00FB ]'
    ##sandbox    
    
    art_vect = re.sub(r' +',' ', re.sub(red,'',re.sub(ur'\u201c',' ',re.sub('\n',' ',doc).lower()))) # Remove the most of the symbols. 
    art_vect = re.sub(',+',',',art_vect) # 
    art_vect = re.sub(', ',',',art_vect) # ,\s\+   remove space after comma, it could be improved
    art_vect = re.sub('^,','',art_vect) # ,\s\+   remove space after comma, it could be improved    
    art_vect = re.sub(r' ','_',art_vect) # Replace the space by underscore.
    art_vect = re.sub(r',$','',art_vect)  # Remove the last comma character. just in case.
    return art_vect

In [8]:
test_text = u',La mansion puprpura de Zimbagüe es la casa, que,,,hemos creado para tí, donde,...'

In [9]:
parse_doc_list_plain(test_text)

u'la_mansion_puprpura_de_zimbag\xfce_es_la_casa,que,hemos_creado_para_t\xed,donde'

In [10]:
talentos_data = []

for pem in pem_data.itertuples():
    data = talento(pem.uid,pem.nombre,pem.ubicacion,pem.id)
    match_la = la_data['uid'] == data.uid
    match_ed = ed_data['uid'] == data.uid
    match_ha = ha_data['uid'] == data.uid
    match_tr = tr_data['uid'] == data.uid    
    data.idiomas = [ idiomas(idioma = lad.idioma, IDI_NIVEL_SP=lad.IDI_NIVEL_SP, 
                             IDI_NIVEL_TE=lad.IDI_NIVEL_TE, IDI_NIVEL_TR=lad.IDI_NIVEL_TR, 
                             IDI_NIVEL_WR=lad.IDI_NIVEL_WR ) for lad in la_data[match_la].itertuples()]
    data.educacion = [educacion(institucion = edu.institucion, area = edu.area_estudio, titulo = edu.titulo) for edu in ed_data[match_ed].itertuples()]
    data.habilidades = [habilidades(habilidades=hab.habilidades) for hab in ha_data[match_ha].itertuples() ]
    data.trabajos = [trabajos(titulo_posicion=tra.posicion,empresa=tra.empresa, 
                              area=tra.area, tipo=tra.tipo, voluntariado = tra.voluntariado,
                              industria = tra.industria ) for tra in tr_data[match_tr].itertuples()]
    talentos_data.append(json.dumps(data,cls=MyEncoder))


In [11]:
talentos_data[2]

'{"trabajos": [{"TRA_IND": "Gobierno", "TRA_TIPO": "Publica", "TRA_EMPRESA": "Secretaria de Economia", "TRA_AREA": "Comercio Internacional", "TRA_POSICION": "Subdirector de derecho internacional"}], "habilidades": [{"HAB_HABILIDAD": "Microsoft Office"}, {"HAB_HABILIDAD": "R"}], "idiomas": [{"IDI_NIVEL_TR": "Ingles", "IDI_NIVEL_WR": "", "IDI_NIVEL_SP": "", "IDI_NOMBRE": "Ingles", "IDI_NIVEL_TE": ""}, {"IDI_NIVEL_TR": "", "IDI_NIVEL_WR": "", "IDI_NIVEL_SP": "", "IDI_NOMBRE": "Espanol", "IDI_NIVEL_TE": "Espanol"}], "uid": "2b111e54-4583-479b-af56-bb05f56c940c", "educacion": [{"EDU_AREA": "Ciencias Sociales", "EDU_TITULO": "Lic. en Derecho", "EDU_INST": "UNAM"}], "ubicacion": "Puebla", "nombre": "Diego Arzamendi", "id": 3}'

### PASO 0 : Preprocesamiento

In [12]:
jlArray = []
for item in jsonlines.Reader(talentos_data).iter():
    preproc = {}    
    preproc[u'ID'] = item[u'id']
    preproc[u'uid'] = item[u'uid']
    preproc[u'INF_UBICACION']=item[u'ubicacion']
    preproc[u'ID_NOMBRE']=item[u'nombre']
    preproc[u'IDI_NOMBRE'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'IDI_NOMBRE']) for cur in item[u'idiomas'] if cur != np.nan ]))
    preproc[u'IDI_NIVEL_SP'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'IDI_NIVEL_SP']) for cur in item[u'idiomas'] if cur != '']))
    preproc[u'IDI_NIVEL_WR'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'IDI_NIVEL_WR']) for cur in item[u'idiomas'] if cur != '']))
    preproc[u'IDI_NIVEL_TR'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'IDI_NIVEL_TR']) for cur in item[u'idiomas'] if cur != '']))
    preproc[u'IDI_NIVEL_TE'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'IDI_NIVEL_TE']) for cur in item[u'idiomas'] if cur != '']))
    preproc[u'EDU_INST'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'EDU_INST']) for cur in item['educacion'] if cur != 'NaN']))
    preproc[u'EDU_TITULO'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'EDU_TITULO']) for cur in item['educacion'] if cur != 'NaN']))
    preproc[u'EDU_AREA'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'EDU_AREA']) for cur in item['educacion'] if cur != 'NaN']))
    preproc['HAB_HABILIDAD'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'HAB_HABILIDAD']) for cur in item['habilidades'] if cur != 'NaN']))
    preproc[u'TRA_EMPRESA'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'TRA_EMPRESA']) for cur in item[u'trabajos'] if cur != 'NaN']))
    preproc[u'TRA_AREA'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'TRA_AREA']) for cur in item[u'trabajos'] if cur != 'NaN']))
    preproc[u'TRA_POSICION'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'TRA_POSICION']) for cur in item[u'trabajos'] if cur != 'NaN']))
    preproc[u'TRA_TIPO'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'TRA_TIPO']) for cur in item[u'trabajos'] if cur != 'NaN']))
    preproc[u'TRA_IND'] = parse_doc_list_plain(','.join([u'{0}'.format(cur[u'TRA_IND']) for cur in item[u'trabajos'] if cur != 'NaN']))
    jlArray.append(preproc)

In [13]:
dfper = pd.DataFrame(jlArray)

In [39]:
dfper.to_csv('talentos.csv')

### PASO 1 : HOMOLOGACION 1

### PASO 2: DEDUPLICACION

### PASO 3: LIMPIEZA

### PASO 4: HOMOLOGACION 2

### PASO 5: DEDUPLICACION

### PASO 6: CLUSTERING